In [1]:
import pyspark as ps
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("yelp_academic") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [173]:
from train_model import str_to_l, if_rest_udf, str_to_list_udf, construct_set_for_ALS

In [146]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [59]:
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import *
from pyspark.sql.functions import udf

In [22]:
from pymongo import MongoClient

mc = MongoClient()
db  = mc['raw_restaurants']
bisn = db['restaurants']
users = db['users']
rv = db['reviews']
rv_s = db['reviews_scrap']

In [23]:
biz = spark.read.json('yelp_dataset/yelp_academic_dataset_business.json')

rev = spark.read.json('yelp_dataset/yelp_academic_dataset_review.json')

user = spark.read.json('yelp_dataset/yelp_academic_dataset_user.json')

<i> Next cell is not need for now </i>

In [24]:
biz.createOrReplaceTempView('business')
rev.createOrReplaceTempView('reviews')

In [27]:
biz.count(), rev.count()

(188593, 5996996)

In [40]:
cities = biz.select('city')
list_of_c = cities.groupBy('city').count().collect()

In [42]:
len(list_of_c)

1111

In [57]:
states = biz.select('state').groupBy('state').count().collect()

In [58]:
biz.select('categories').take(2)

[Row(categories='Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel'),
 Row(categories='Chicken Wings, Burgers, Caterers, Street Vendors, Barbeque, Food Trucks, Food, Restaurants, Event Planning & Services')]

In [62]:
str_to_l('Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel')

['Tours', 'Breweries', 'Pizza', 'Restaurants', 'Food', 'Hotels & Travel']

In [69]:
str_to_list_udf = udf(str_to_l, ArrayType(StringType()))

In [77]:
rests = biz.filter(if_rest_udf(biz.categories))

In [101]:
rests.cache()

DataFrame[address: string, attributes: struct<AcceptsInsurance:string,AgesAllowed:string,Alcohol:string,Ambience:string,BYOB:string,BYOBCorkage:string,BestNights:string,BikeParking:string,BusinessAcceptsBitcoin:string,BusinessAcceptsCreditCards:string,BusinessParking:string,ByAppointmentOnly:string,Caters:string,CoatCheck:string,Corkage:string,DietaryRestrictions:string,DogsAllowed:string,DriveThru:string,GoodForDancing:string,GoodForKids:string,GoodForMeal:string,HairSpecializesIn:string,HappyHour:string,HasTV:string,Music:string,NoiseLevel:string,Open24Hours:string,OutdoorSeating:string,RestaurantsAttire:string,RestaurantsCounterService:string,RestaurantsDelivery:string,RestaurantsGoodForGroups:string,RestaurantsPriceRange2:string,RestaurantsReservations:string,RestaurantsTableService:string,RestaurantsTakeOut:string,Smoking:string,WheelchairAccessible:string,WiFi:string>, business_id: string, categories: string, city: string, hours: struct<Friday:string,Monday:string,Saturday:string

In [79]:
#rests = rests.withColumn('categories', str_to_list_udf(rests.categories))

In [93]:
##Wanna try two ways
rests.createOrReplaceTempView('rests')
rev.createOrReplaceTempView('reviews')

In [91]:
rest_id = rests.select('business_id')

In [97]:
rest_id.createOrReplaceTempView('rests_id')

In [98]:
%%time
select = '''
SELECT  reviews.* 
FROM rests_id 
LEFT JOIN reviews ON rests_id.business_id == reviews.business_id


'''
rest_rev = spark.sql(select)
rest_rev.show(5)

+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|--9e1ONYQuAa-CB_R...|   0|2014-02-25|    0|8eEp26oqTXmuIjVUJ...|    3|It was a real up ...|     0|AdxhlxXSGjRsCQxG6...|
|--9e1ONYQuAa-CB_R...|   0|2017-02-14|    0|VETXTwMw6qxzOVDlX...|    5|went for dinner t...|     0|ymlnR8UeFvB4FZL56...|
|--9e1ONYQuAa-CB_R...|   0|2013-12-19|    0|NQt645IJwa5ADM26U...|    5|we are a repeat c...|     0|TMqFvYbWqs8BnjLsE...|
|--9e1ONYQuAa-CB_R...|   0|2013-12-02|    2|G9TSEZ8BfuSmcf5xy...|    3|Thanksgiving dinn...|     5|KmubWHd-t2d3dxLZ1...|
|--9e1ONYQuAa-CB_R...|   0|2015-01-01|    0|6prMgAjp2LZjuxRvm...|    4|We were seated pr...|     0|bMFqD4beGrt_LjNGG...|
+--------------------+----+-----

In [96]:
%%time
rest_rev2 = rest_id.join(rev, "business_id")
rest_rev2.show(5)

+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|iCQpiavjjPzJ5_3gP...|   0|2011-02-25|    0|x7mDIiDB3jEiPGPHO...|    2|The pizza was oka...|     0|msQe1u7Z_XuqjGoqh...|
|pomGBqfbxcqPv14c3...|   0|2012-11-13|    0|dDl8zu1vWPdKGihJr...|    5|I love this place...|     0|msQe1u7Z_XuqjGoqh...|
|jtQARsP6P-LbkyjbO...|   1|2014-10-23|    1|LZp4UX5zK3e-c5ZGS...|    1|Terrible. Dry cor...|     3|msQe1u7Z_XuqjGoqh...|
|elqbBhBfElMNSrjFq...|   0|2011-02-25|    0|Er4NBWCmCD4nM8_p1...|    2|Back in 2005-2007...|     2|msQe1u7Z_XuqjGoqh...|
|Ums3gaP2qM3W1XcA5...|   0|2014-09-05|    0|jsDu6QEJHbwP2Blom...|    5|Delicious healthy...|     0|msQe1u7Z_XuqjGoqh...|
+--------------------+----+-----

### Join faster??? after lunch try in diff order

In [99]:
rest_rev2.count()

3654797

In [102]:
rest_rev2.cache()

DataFrame[business_id: string, cool: bigint, date: string, funny: bigint, review_id: string, stars: bigint, text: string, useful: bigint, user_id: string]

Baseline --- predict just mean rest rating

In [109]:
base_pred = rests.select('business_id', 'stars').withColumnRenamed('stars',
'mean_rating').join(rev.select('business_id', 'stars'), 'business_id')

In [114]:
baseline_rdd =base_pred.rdd


In [119]:
r1 = baseline_rdd.first()
r1[0], r1[1],r1[2]

('iCQpiavjjPzJ5_3gPD5Ebg', 4.0, 2)

In [118]:
baseline_rdd.cache()

MapPartitionsRDD[263] at javaToPython at NativeMethodAccessorImpl.java:0

In [120]:
sum_sq = baseline_rdd.map(lambda x : (x[2] - x[1])**2).sum()
num = baseline_rdd.count()


In [138]:
from math import sqrt
sum_sq/num, sqrt(sum_sq/num)

(1.5400588596302338, 1.240991079593336)

In [164]:
user_id = rest_rev2.select('user_id').groupBy('user_id').count().selectExpr('user_id', 'count AS user_count')

In [163]:
rest_rev2.select('user_id').groupBy('user_id').count().selectExpr('user_id', 'count AS user_count').take(2)

[Row(user_id='Cxv3SNudPrCavvHJfIbwhQ', user_count=2),
 Row(user_id='pQY_h122h0nzPn5l5OY9nQ', user_count=1)]

In [128]:
user_id.first()

Row(user_id='Cxv3SNudPrCavvHJfIbwhQ', count=2)

In [129]:
user_id.cache()

DataFrame[user_id: string, count: bigint]

In [130]:
user_c_df =user_id.toPandas()

In [136]:
user_c_df.columns, user_c_df.shape

(Index(['user_id', 'count'], dtype='object'), (1031840, 2))

In [135]:
rest_id_df = rest_id.toPandas()
rest_id_df.shape

(57173, 1)

In [147]:
(user_c_df['count']>10).sum(), user_c_df['count'][user_c_df['count']>10].sum()

(56425, 1611480)

In [165]:
def look_how_dence_rating_matrix(rev, m):
    rating = rev.select('user_id', 'business_id', 'stars')
    user_id = rating.select('user_id').groupBy('user_id').count().selectExpr('user_id', 'count AS user_count')
    
    filt_rat = user_id.filter(user_id.user_count>m).join(rating, 'user_id')
    u  = user_id.filter(user_id.user_count>m).count()
    v = filt_rat.select('business_id').groupBy('business_id').count().count()
    r = filt_rat.count()
    k = r/(u+v)
    return u, v, r, k

In [166]:
look_how_dence_rating_matrix(rest_rev2,10)

(56425, 55341, 1611480, 14.418338313977417)

In [167]:
%%time
m_k = {}
for m in range(20):
    m_k[m] = look_how_dence_rating_matrix(rest_rev2,m)

CPU times: user 496 ms, sys: 184 ms, total: 680 ms
Wall time: 16min 26s


In [168]:
m_k

{0: (1031840, 57173, 3654797, 3.356063701718896),
 1: (464515, 57049, 3087472, 5.919641693061638),
 2: (292072, 56861, 2742586, 7.859921532213921),
 3: (207596, 56683, 2489158, 9.41867496093144),
 4: (157816, 56483, 2290038, 10.686181456749681),
 5: (125431, 56287, 2128113, 11.71107430194037),
 6: (103142, 56090, 1994379, 12.52498869573955),
 7: (86606, 55915, 1878627, 13.181404845601701),
 8: (74139, 55737, 1778891, 13.696841602759555),
 9: (64410, 55537, 1691330, 14.100644451299324),
 10: (56425, 55341, 1611480, 14.418338313977417),
 11: (50120, 55161, 1542125, 14.647704714050969),
 12: (44882, 55003, 1479269, 14.80972117935626),
 13: (40438, 54841, 1421497, 14.919310656073216),
 14: (36842, 54667, 1371153, 14.98380487165197),
 15: (33691, 54504, 1323888, 15.010918986337094),
 16: (30959, 54338, 1280176, 15.008452817801329),
 17: (28556, 54197, 1239325, 14.976194216523872),
 18: (26490, 54054, 1202137, 14.925220997218911),
 19: (24724, 53929, 1168583, 14.857449811196012)}

In [170]:
spark.createDataFrame(user_c_df).printSchema()

root
 |-- user_id: string (nullable = true)
 |-- count: long (nullable = true)
 |-- user_num: long (nullable = true)



In [159]:
user_id.columns  

['user_id', 'count']

In [148]:
user_c_df['user_num'] = user_c_df.index

In [149]:
user_id2 = sc.

,user_id,count,user_num
0,Cxv3SNudPrCavvHJfIbwhQ,2,0
1,pQY_h122h0nzPn5l5OY9nQ,1,1
2,iL42nhZyqGGNuX9Vj379KA,1,2
3,wyTsy3kiFhXuaCsxD9ku6Q,2,3
4,3pQxT_d2seD4p7hTLZ0uSQ,7,4


In [172]:
train_set = construct_set_for_ALS(rest_rev2, 9)

AttributeError: 'DataFrame' object has no attribute '_jdf'